# RNNレイヤの実装

In [4]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.functions import sigmoid

ローカル環境での実行です


### [演習]
* 以下のRNNレイヤのクラスを完成させましょう

In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        """
        
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        """
        順伝播計算
        """
        Wx, Wh, b = self.params
        
        # 行列の積　+　行列の積 + バイアス
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        
        # 活性化関数に入れる
        h_next = np.tanh(t)

        # 値の一時保存
        self.cache = (x, h_prev, h_next)
        
        return h_next

    def backward(self, dh_next):
        """
        逆伝播計算
        """
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        # tanhでの逆伝播
        A3 = dh_next * (1 - h_next ** 2) # dh_next * (1 - y^2)
        
        # バイアスbの勾配
        # Nの方向に合計する
        db = np.sum(A3, axis=0)
        
        # 重みWhの勾配
        dWh = np.dot(h_prev.T, A3)
        
        # 1時刻前に渡す勾配
        dh_prev = np.dot(A3, Wh.T)
        
        # 重みWxの勾配
        dWx = np.dot(x.T, A3)
        
        # 入力xに渡す勾配
        dx = np.dot(A3, Wx.T)

        # 勾配をまとめる
        self.grads[0][:] = dWx # 同じメモリ位置に代入
        self.grads[1][:] = dWh # 同じメモリ位置に代入
        self.grads[2][:] = db # 同じメモリ位置に代入

        return dx, dh_prev

In [6]:
D = 1 # 入力層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
rnn = RNN(Wx, Wh, b)
print("id of rnn.grads[0]", id(rnn.grads[0]))
print("id of rnn.grads[1]", id(rnn.grads[1]))
print("id of rnn.grads[2]", id(rnn.grads[2]))
print()

id of rnn.grads[0] 140242226875632
id of rnn.grads[1] 140242226876016
id of rnn.grads[2] 140242226874384



In [8]:
# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
h_next = rnn.forward(x, h_prev)
print("h_next=", h_next)
print()

h_next= [[-0.89382831  0.96851345 -0.96058241  0.99779854 -0.68497571]
 [ 0.59453191 -0.90462357  0.87601757  0.5474022   0.96721127]
 [-0.99803263 -0.04496816  0.75087349  0.99925754 -0.59154562]
 [-0.81780563  0.99896125  0.93028599  0.98759778  0.7456259 ]]



In [9]:
# 逆伝播計算
dh_next = np.random.randn(N, H )
dx, dh_prev = rnn.backward(dh_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()

print("id of rnn.grads[0]", id(rnn.grads[0]))
print("id of rnn.grads[1]", id(rnn.grads[1]))
print("id of rnn.grads[2]", id(rnn.grads[2]))
print()

dx= [[3.47757136]
 [1.8550463 ]
 [0.68981227]
 [0.86816681]]

dh_prev= [[ 1.94073061  0.44613561 -2.53054981  1.81056844 -0.94559005]
 [ 1.49533747  0.28214455 -0.18134925  0.14886095  0.38772888]
 [-1.16792788 -0.98316478  0.03568121  0.87811387  0.14401074]
 [ 0.19647807  0.41057318 -0.13819291  0.42770633 -0.21434265]]

id of rnn.grads[0] 140242226875632
id of rnn.grads[1] 140242226876016
id of rnn.grads[2] 140242226874384

